In [1]:
## Idea
# 1. The idea is to record voice for all the words in a text corpus and play each words for a new given 
# sequence of text. 
# 2. Each word is governed by a pronounciation vector that contains the way to pronouce the word. That means 
# how to change a few parameters of that small audio chunk (pre-recorded word) so that the new word would match
# the pronounciations of the sequence and would sound like real human speaking.

# The process is going to be carried out for Hindi Language and all the words is going to spoken by myself.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf
import librosa
from IPython.display import Audio
import random


2022-01-19 14:21:53.510832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-19 14:21:53.510880: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Here are the examples of a few sentece that will build up our text-corpus. 

# 1. d m i faeneans me ap ka swagat hae .
# 2. lon sambandhit jankari ke liye ek dabaen .
# 3. naya khata khol ne ke liye do dabaen .
# 4. hamare grahak seva pratinidhi se baat karne ke liye teen dabaen .
# 5. yatri gan kripaya dhyan den , gadi sankshaya sat panch nau ek sunya do , hazrat nizanumdin express, 
# pletform number char par thode hi der men aa rahi hae .


In [4]:
# Now let's find all the unique words in the text corpus.

text_corpus = '''
 d m i faeneans me ap ka swagat hae . 
 lon sambandhit jankari ke liye ek dabaen . 
 naya khata khol ne ke liye do dabaen . 
 hamare grahak seva pratinidhi se baat karne ke liye teen dabaen .
 yatri gan kripaya dhyan den , gadi sankshaya sat panch nau ek sunya do , hazrat nizanumdin express , 
 pletform number char par thode hi der men aa rahi hae .
'''
print(text_corpus)



 d m i faeneans me ap ka swagat hae . 
 lon sambandhit jankari ke liye ek dabaen . 
 naya khata khol ne ke liye do dabaen . 
 hamare grahak seva pratinidhi se baat karne ke liye teen dabaen .
 yatri gan kripaya dhyan den , gadi sankshaya sat panch nau ek sunya do , hazrat nizanumdin express , 
 pletform number char par thode hi der men aa rahi hae .



In [5]:
# Now we need to remove any new line.
text_corpus_m = text_corpus.replace('\n', '')
# print(text_corpus_m)

# Converting all the text to a list.
text_corpus_list = text_corpus_m.split(' ')
# print(text_corpus_list)

# Finding the unique words.
text_corpus_set = set(text_corpus_list)
print(len(text_corpus_set))
print(text_corpus_set)

# Here an unique thing is that comman ',', period '.' and blank space ' ' are also considered as individual
# words and having their own voice chinks assigned to them.


56
{'', 'khol', 'hamare', 'ke', 'express', 'pletform', 'yatri', 'char', 'liye', 'khata', 'ne', 'den', 'nizanumdin', 'men', 'do', 'dhyan', 'aa', 'teen', 'jankari', 'sankshaya', 'der', 'pratinidhi', 'ka', 'swagat', 'faeneans', 'i', 'ap', '.', 'sambandhit', 'panch', 'number', 'grahak', 'nau', 'seva', 'd', 'me', 'rahi', 'm', 'hae', ',', 'ek', 'sat', 'thode', 'dabaen', 'gadi', 'sunya', 'lon', 'se', 'hazrat', 'naya', 'hi', 'gan', 'kripaya', 'karne', 'par', 'baat'}


In [6]:
# Now all we need to do is that record all these 56  words and by self and save all these in folder.

# Let's save all these words in a text file. 
# f = open('words.txt', 'w')

# for index, word in enumerate(text_corpus_set):
#     f.write(str(index) + '. ' + word + '\n')
   
# f.close()


In [7]:
# Now let's open our text file and read all the words.
f = open('words.txt', 'r')
f_content = f.read()
print(f_content)


0.  
1. hae
2. liye
3. karne
4. gadi
5. se
6. express
7. thode
8. ek
9. do
10. dhyan
11. baat
12. m
13. den
14. ,
15. hi
16. pletform
17. panch
18. khol
19. rahi
20. me
21. jankari
22. sankshaya
23. seva
24. number
25. pratinidhi
26. men
27. kripaya
28. d
29. grahak
30. .
31. sambandhit
32. hazrat
33. ne
34. ke
35. khata
36. lon
37. ka
38. teen
39. par
40. sunya
41. sat
42. der
43. nau
44. dabaen
45. naya
46. aa
47. yatri
48. swagat
49. gan
50. faeneans
51. ap
52. i
53. nizanumdin
54. hamare
55. char



In [8]:
# Now it's time split all the words and save them as index.
# A full stop is going to be 500 ms. A comma is going to be 250 ms. a blank space is going to be 100 ms.

# Now that we have all the voice chunks of all different words. Let's import all of those our Jupyter Notebook 
# environment. 
file_list = os.listdir('data/words/')
print(len(file_list))
print(file_list)


56
['53.wav', '33.wav', '25.wav', '40.wav', '20.wav', '8.wav', '37.wav', '50.wav', '4.wav', '3.wav', '0.wav', '13.wav', '47.wav', '49.wav', '28.wav', '23.wav', '43.wav', '42.wav', '15.wav', '17.wav', '6.wav', '52.wav', '19.wav', '22.wav', '36.wav', '35.wav', '24.wav', '46.wav', '38.wav', '55.wav', '30.wav', '34.wav', '16.wav', '1.wav', '2.wav', '12.wav', '29.wav', '41.wav', '26.wav', '18.wav', '45.wav', '10.wav', '39.wav', '27.wav', '11.wav', '7.wav', '31.wav', '14.wav', '51.wav', '9.wav', '21.wav', '44.wav', '48.wav', '54.wav', '32.wav', '5.wav']


In [40]:
# Now importing all the audio files to the env using Librosa in terms of Numpy arrays.
# Before that we will make a testing on just one file and this testing is going to be used for doing anything 
# using Librosa.
ar, sr = librosa.load('data/words/6.wav')

# The shape shows that this is a mono audio.
print(ar.shape)
print(sr)

# Now applying pitch change to the array
pitch_shifted_arr = librosa.effects.pitch_shift(ar, sr, n_steps=-4)

# print(ar)
print(pitch_shifted_arr)

# Let's play the audio arr.
Audio(data=pitch_shifted_arr, rate=sr)



(11139,)
22050
[ 0.01413357  0.01906158  0.01641738 ...  0.02804443 -0.00179333
  0.        ]


In [12]:
# Now let's import all the files as a key value pair in a dict where the key is going to be the name of the file
# and the value is goin to be the Numpy array of the audio file.
words_dict = {}

for file_name in file_list:
    inst_arr, inst_sr = librosa.load('data/words/{}'.format(file_name))
    words_dict[file_name.split('.')[0]] = inst_arr
    
display(words_dict)


{'53': array([-0.05807065, -0.0730717 , -0.0657716 , ...,  0.03283238,
         0.03596155,  0.        ], dtype=float32),
 '33': array([-0.03623761, -0.05612076, -0.05116033, ...,  0.00607129,
         0.00617177,  0.        ], dtype=float32),
 '25': array([-0.0088812 , -0.01927411, -0.02374777, ...,  0.0247072 ,
         0.02594318,  0.02609218], dtype=float32),
 '40': array([ 0.02933602,  0.01773386,  0.0283764 , ..., -0.04408102,
        -0.04447043,  0.        ], dtype=float32),
 '20': array([-0.04016411, -0.05384111, -0.04711616, ..., -0.03825574,
        -0.04059773,  0.        ], dtype=float32),
 '8': array([ 0.00196469, -0.00086853, -0.00220581, ...,  0.01015405,
         0.01889697,  0.        ], dtype=float32),
 '37': array([-0.01834474, -0.03376151, -0.0319236 , ..., -0.06065378,
        -0.06118362, -0.06896183], dtype=float32),
 '50': array([ 0.02171628,  0.01717837,  0.02399947, ..., -0.03661504,
         0.00515745, -0.02106968], dtype=float32),
 '4': array([-0.01700076,

In [51]:
# Concatinating the audio file is going to be very easy. It's just like concatinating the numpy arrays.

# sentence = 'yatri| |gan| |kripaya| |dhyan| |den|.'
# sentence = 'yatri gan kripaya dhyan den , , , gadi sankshaya , sat panch nau ek sunya do , , , hazrat nizanumdin ' \
# + 'express , , , pletform number , char , par , thode hi der men , , , aa rahi hae .'

sentence = 'd m i faeneans me , , , ap ka , , , swagat hae .'

# Now we need to convert all the words to repective number in text form.
sentence_list = sentence.split(' ')
print(sentence_list)

print('')

# Now we need to convert this array to the number index.
# To do that we are going to use the word.txt file. 
# Let's convert the f_content (the word.txt file) to a dict.
word_index_dict = {' ': '0'}

f_conent_words = f_content.split('\n')
# print(f_conent_words)

for i in range(len(f_conent_words) - 1):
    if i != 0:
        inst_element = f_conent_words[i]
        # print(inst_element)
        inst_name = inst_element.split('. ')[1]
        # print(inst_name)
        inst_index = inst_element.split('. ')[0]
        # print(inst_index)
        word_index_dict[inst_name] = inst_index
        
print(word_index_dict)


['d', 'm', 'i', 'faeneans', 'me', ',', ',', ',', 'ap', 'ka', ',', ',', ',', 'swagat', 'hae', '.']

{' ': '0', 'hae': '1', 'liye': '2', 'karne': '3', 'gadi': '4', 'se': '5', 'express': '6', 'thode': '7', 'ek': '8', 'do': '9', 'dhyan': '10', 'baat': '11', 'm': '12', 'den': '13', ',': '14', 'hi': '15', 'pletform': '16', 'panch': '17', 'khol': '18', 'rahi': '19', 'me': '20', 'jankari': '21', 'sankshaya': '22', 'seva': '23', 'number': '24', 'pratinidhi': '25', 'men': '26', 'kripaya': '27', 'd': '28', 'grahak': '29', '.': '30', 'sambandhit': '31', 'hazrat': '32', 'ne': '33', 'ke': '34', 'khata': '35', 'lon': '36', 'ka': '37', 'teen': '38', 'par': '39', 'sunya': '40', 'sat': '41', 'der': '42', 'nau': '43', 'dabaen': '44', 'naya': '45', 'aa': '46', 'yatri': '47', 'swagat': '48', 'gan': '49', 'faeneans': '50', 'ap': '51', 'i': '52', 'nizanumdin': '53', 'hamare': '54', 'char': '55'}


In [52]:
# Now that we are having the dict we can do a vectorization.
sentence_vector = []

for ele in sentence_list:
    inst_index = word_index_dict[ele]
    sentence_vector.append(inst_index)
    
print(sentence_vector)


['28', '12', '52', '50', '20', '14', '14', '14', '51', '37', '14', '14', '14', '48', '1', '30']


In [53]:
# Now we need to do a concatination of the numpy arrays according to the vectors.
final_array = np.array([])
scale = [-3, -2, -1, 0, 1, 2, 3, 4, 5, 6]

for index in sentence_vector:
    inst_numpy_ar = words_dict[index]
    
    # Pitch shirfter
    # --------
    rand_int = random.randint(0, 2)
    tone = scale[rand_int]
    inst_numpy_ar = librosa.effects.pitch_shift(inst_numpy_ar, sr, n_steps=tone)
    # --------
    
    final_array = np.concatenate([final_array, inst_numpy_ar[int(sr/8000): len(inst_numpy_ar)-(int(sr/8000))]])
    
print(final_array.shape)

# Let's play the final array.

print(sentence)

Audio(data=final_array, rate=sr)

# Saving the audio.
#

(102294,)
d m i faeneans me , , , ap ka , , , swagat hae .


In [17]:
# So what are the things we need to do here.
# 1. We need 5 different types of speaking emotions - Welcoming, Neutral, Happy, Confusion, Excitement.
# 2. The pronounciation vector is going to be 4 dimenstional - gap_before, speed, pitch, volume

# So for each word we need a 4 dimentional pronounciation vector.

# We need 5 different presents each for one emotion. The present should be long enough to carry out a 
# a talking for 15 seconds and it would repeat the same. That's it.  

# A final word. We can just take all the pre-recordings for that by speaking all the sentences and use them 
# to say different things. 
